## 1. Preamble
Import source code and packages

In [1]:
import requests
import re
import string
import json
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint

def collectSoup(url):
    ua = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36'}
    page = requests.get(url,headers = ua)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

def getJobData(url):
    
    soup = collectSoup(url)
    
    schema = soup.find("script",id="jobPostingSchema") #Retrieve the job listing schema
    jsonData = json.loads(schema.string) #Convert to JSON Object.
    
    relatedjobtags = soup.find(class_="relatedJobsList").find_all("a") #Retrieve the list of related job searches
    relatedjobs = [(i.string.replace(' jobs',''),i['href']) for i in relatedjobtags if not('in-' in i['href']) and not (i.string is None)] #Retrive pairs of (job name[minus 'jobs'], suburl)
    jsonData['relatedjobs'] = relatedjobs #Add related jobs to the JSON object
    
    return jsonData


In [2]:
check = getJobData('https://www.totaljobs.com/job/accounting-technician/trial-balance-consulting-job85291461')

In [3]:
mainurl = 'https://www.totaljobs.com/'

main = collectSoup(mainurl)

catlistfull = main.find(class_='tab-content sector-tab-content tab-pane fade active in').find_all('a')

catlist = [(cat['href'],cat.string) for cat in catlistfull]


In [4]:
#caturl1 = mainurl+catlist[1][0][1:]

mainurl = 'https://www.totaljobs.com/jobs/data-analyst'
suburls = ['?page='+str(n) for n in list(range(2,6))]
suburls = ['',*suburls]
urls = [mainurl+suburl for suburl in suburls]

print(urls)

['https://www.totaljobs.com/jobs/data-analyst', 'https://www.totaljobs.com/jobs/data-analyst?page=2', 'https://www.totaljobs.com/jobs/data-analyst?page=3', 'https://www.totaljobs.com/jobs/data-analyst?page=4', 'https://www.totaljobs.com/jobs/data-analyst?page=5']


In [5]:
soups = [collectSoup(url) for url in urls]

In [6]:
joburlslist = []

for soup in soups:
    
    jobsonpage = soup.find_all(class_="job-title")
    joburls = [job.find("a")['href'] for job in jobsonpage]
    joburlslist = [*joburlslist, *joburls]

pprint(len(joburlslist))

100


In [7]:
jobdata = [getJobData(url) for url in joburlslist]

In [8]:
jobtitles = [job['title'] for job in jobdata]

print(jobtitles)

['Data Analyst', 'Data Analyst', 'Data Analyst, Agile, News Media', 'Energy Data Analyst', 'Data Analyst', 'Data Analyst - SAS', 'Data Analyst - SAS', 'Data Analyst (SQL / T-SQL) - Kensington', 'Data Analyst (Excel) - Bristol - New Role - Exclusive', 'Data Analyst', 'Data Analyst', 'Trade / Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst /Business Analyst (Reference Data) - Banking background', 'Data Analyst - ETL - London - up to £60,000', 'Data Analyst', 'Data Analyst - ETL - London - up to £60,000', 'Data Analyst', 'Data Analyst', 'Data Analyst - SQL - SSRS / SSIS - OLAP - BI - Azure - AWS', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analysts', 'Data Analyst - Marketing', 'Data Analyst', 'Data Analyst', 'Administrator/ Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst - Fundraising', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data A

## Saving Data
Connecting to a MongoDB database on learn-v1 (Orowa's server)

In [9]:
from pymongo import MongoClient
client = MongoClient('209.97.135.177:27017')
db = client.jobs

In [ ]:
result = db.jobs.insert_many(jobdata)
pprint (result.inserted_ids)

In [10]:
query = db.jobs.find_one({'@type': 'JobPosting'})
pprint (query)

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 '_id': ObjectId('5c70dc6f5e95262ab42c3fdf'),
 'datePosted': '2019-02-09T11:00+00:00',
 'description': '<p><strong>Finance Data Analyst <br></strong><strong>London '
                '<br></strong><strong>£40,000 + Benefits </strong></p><p>A '
                'successful Media company are looking for a Finance Data '
                'Analyst to join them in the heart of London. This is a tech '
                'savvy business, with an excellent reputation, now looking to '
                'add to their team. You will be reporting into the Head of '
                'Finance gathering requirements and delivering insight '
                'solutions on client projects, using a mixture of SQL and '
                'Python (training provided). You will be tasked with '
                'identifying opportunities for deeper analysis, producing '
                'reports and insight and really using data to answer business '
            

In [ ]:
#db.jobs.drop()